In [1]:
import os
import numpy as np
import logging

from asgardpy.analysis import AsgardpyAnalysis
from asgardpy.config import AsgardpyConfig
from asgardpy.stats import (
    check_model_preference_aic,
    fetch_all_analysis_fit_info,
    get_model_config_files,
    tabulate_best_fit_stats,
    copy_target_config,
)

In [2]:
# os.environ['GAMMAPY_DATA']

In [3]:
log = logging.getLogger("Check Preferred Spectral Model")

In [4]:
config_file = "../src/asgardpy/tests/config_test_base.yaml"

# Initiate the main "base" config

In [5]:
main_config = AsgardpyConfig.read(config_file)
target_source_name = main_config.target.source_name

In [6]:
spec_model_temp_files = get_model_config_files(["lp", "bpl", "ecpl", "pl", "eclp", "sbpl"])

In [7]:
main_config.dataset3d.instruments[0].input_dl3[0].input_dir = f"{os.environ['GAMMAPY_DATA']}fermipy-crab/"
main_config.dataset3d.instruments[0].input_dl3[1].input_dir = f"{os.environ['GAMMAPY_DATA']}fermipy-crab/"

main_config.dataset1d.instruments[0].input_dl3[0].input_dir = f"{os.environ['GAMMAPY_DATA']}hess-dl3-dr1/"
main_config.dataset1d.instruments[1].input_dl3[0].input_dir = f"{os.environ['GAMMAPY_DATA']}magic/rad_max/data/"

In [8]:
def fetch_all_analysis_objects(main_config, spec_model_temp_files):
    """For a list of spectral models, initiate AsgardpyAnalysis objects."""
    main_analysis_list = {}
    spec_models_list = []

    for temp in spec_model_temp_files:
        temp_model = AsgardpyAnalysis(main_config)
        temp_model.config.target.models_file = temp

        temp_model_2 = AsgardpyAnalysis(temp_model.config)

        copy_target_config(temp_model, temp_model_2)

        spec_tag = temp.name.split(".")[0].split("_")[-1]
        spec_models_list.append(spec_tag)
        main_analysis_list[spec_tag] = {}

        main_analysis_list[spec_tag]["Analysis"] = temp_model_2

    spec_models_list = np.array(spec_models_list)

    return main_analysis_list, spec_models_list

In [9]:
%%time
main_analysis_list, spec_models_list = fetch_all_analysis_objects(
    main_config, spec_model_temp_files
)

Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filem

CPU times: user 56.2 ms, sys: 428 µs, total: 56.7 ms
Wall time: 55.5 ms


In [10]:
spec_models_list

array(['lp', 'bpl', 'ecpl', 'pl', 'eclp', 'sbpl'], dtype='<U4')

# Run Analysis steps

In [11]:
%%time
PL_idx = 0

for i, tag in enumerate(spec_models_list):
    log.info("Spectral model being tested: %s", tag)

    main_analysis_list[tag]["Analysis"].run()

    if tag == "pl":
        PL_idx = i


Spectral model being tested: lp
Perform DL3 to DL4 process!
1 number of 3D Datasets given
The different keys used: 00 01
Selecting the file with name containing 00
Selecting the file with name containing 00
Selecting the file with name containing 00
Selecting the file with name containing 00
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set MJD-OBS to 58484.000000 from DATE-OBS.
Set MJD-END to 59729.988334 from DATE-END'. [astropy.wcs.wcs]
Set MJD-OBS to 58484.000000 from DATE-OBS.
Set MJD-END to 59729.988334 from DATE-END'.
Using the exclusion mask to create a safe mask
Selecting the file with name containing 01
Selecting the file with name containing 01
Selecting the file with name containing 01
Selecting the file with name containing 01
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set MJD-OBS to 58484.000000 from DATE-OBS.
Set MJD-END to 59

CPU times: user 6min 59s, sys: 51.3 s, total: 7min 51s
Wall time: 10min 26s


# Get all the Fit results

In [12]:
%%time
fit_success_list, stat_list, dof_list, pref_over_pl_chi2_list = fetch_all_analysis_fit_info(
    main_analysis_list, spec_models_list
)

CPU times: user 1.07 ms, sys: 0 ns, total: 1.07 ms
Wall time: 1.18 ms


In [13]:
# If any spectral model has at least 5 sigmas preference over PL
best_sp_idx_lrt = np.nonzero(pref_over_pl_chi2_list == np.nanmax(pref_over_pl_chi2_list))[0]
for idx in best_sp_idx_lrt:
    if pref_over_pl_chi2_list[idx] > 5:
        sp_idx_lrt = idx
        log.info("Best preferred spectral model over PL is %s", spec_models_list[idx])
    else:
        sp_idx_lrt = PL_idx
        log.info("No other model preferred over PL")


Best preferred spectral model over PL is ecpl


In [14]:
list_rel_p = check_model_preference_aic(stat_list, dof_list)

In [15]:
best_sp_idx_aic = np.nonzero(list_rel_p == np.nanmax(list_rel_p))[0]

for idx in best_sp_idx_aic:
    if list_rel_p[idx] > 0.95:
        sp_idx_aic = idx
        log.info("Best preferred spectral model is %s", spec_models_list[fit_success_list][idx])
    else:
        sp_idx_aic = PL_idx
        log.info("No other model preferred, hence PL is selected")

No other model preferred, hence PL is selected


# Write the results into a Table

In [16]:
stats_table = tabulate_best_fit_stats(spec_models_list, fit_success_list, main_analysis_list, list_rel_p)

stats_table.meta["Target source name"] = target_source_name

In [17]:
stats_table

Spectral Model,TS of Best Fit,TS of Max Fit,TS of Goodness of Fit,DoF of Fit,Significance ($\sigma$) of Goodness of Fit,p-value of Goodness of Fit,Pref over PL (chi2),Pref over PL (p-value),Pref over PL (DoF),Relative p-value (AIC)
str4,float64,float64,float64,int64,float64,float64,float64,float64,int64,float64
LP,-4442.086,-4520.08,77.994,72,1.049,0.2941,6.984,2.866e-12,1,0.09067
BPL,-4393.153,-4520.08,126.927,72,3.977,6.987e-05,0.0,1.0,1,2.147e-12
ECPL,-4442.588,-4520.08,77.493,72,1.02,0.3079,7.02,2.219e-12,1,0.1165
PL,-4393.308,-4520.08,126.773,73,3.895,9.8e-05,0.0,0.0,0,8.533e-13
ECLP,-4444.423,-4520.08,75.658,71,0.973,0.3306,6.839,7.953e-12,2,0.7928
SBPL,-4206.942,-4520.08,313.138,70,11.864,1.812e-32,0.0,1.0,3,5.824e-52
